In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_core.tools import create_retriever_tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

from dotenv import load_dotenv
import os
load_dotenv()

c:\Users\aryan\Desktop\LangChain Projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


True

### Tool Creation

In [2]:
# tool for wikipedia -- general knowledge
api_wrapper_wiki = WikipediaAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=1000,
    # responsible for sending the http call, parsing the response, and returning the
    # text and the metadata
)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
# this function makes it into a tool. Responsible for accepting natural language, calling
# the configured api wrapper, and then returning results in a format the agent can use
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\aryan\\Desktop\\LangChain Projects\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=1000))

In [3]:
# tool for arxiv -- research papers
api_wrapper_arxiv = ArxivAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=1000,
)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv_tool.name

'arxiv'

In [4]:
tools = [wiki_tool, arxiv_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\aryan\\Desktop\\LangChain Projects\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=1000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000))]

In [5]:
# Custome tools -- RAG tools
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Deep_space_exploration")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="DeepSpaceExplorationRetriever",
    description="Useful for when you need to find information about deep space exploration from Wikipedia articles.",
)

tools.append(retriever_tool)
retriever_tool.name

'DeepSpaceExplorationRetriever'

In [18]:
# running the tools in an agent
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000222F9093C50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000222F91A47D0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
# prompt template
from langchain_classic import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
# creating agents
from langchain_classic.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [21]:
# agent executer
from langchain_classic.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [22]:
agent_executor.invoke(
    {
        "input": "Tell me about deep space exploration"
    }
)



> Entering new AgentExecutor chain...

Invoking: `DeepSpaceExplorationRetriever` with `{'query': 'deep space exploration'}`


Research in deep space is ongoing and rapidly developing. In 2011, after the retirement of the Space Shuttle, NASA announced its intentions to invest money into developing three technologies vital to deep space exploration. The "must-have technologies" include a deep space atomic clock, a large solar sail and a more advanced laser communications system to improve communication, navigation, and propulsion in future missions.[10] In June 2013, NASA announced the selection of eight American astronauts that will begin to train for future deep space missions beyond low Earth orbit. NASA intends that these eight astronauts to train for future Mars or asteroid travel.[11]

At present the farthest space probe humankind has constructed and launched from Earth is Voyager 1, which was announced on December 5, 2011,[2] to have reached the outer edge of the Solar System,[3

{'input': 'Tell me about deep space exploration',
 'output': 'The current farthest space probe in our universe is Voyager 1, which was announced on December 5, 2011, to have reached the outer edge of the Solar System, and entered interstellar space on August 25, 2012.'}

In [24]:
agent_executor.invoke(
    {
        "input": "Tell me about a recipe to make cookies"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Cookie recipe'}`


Page: Chocolate chip cookie
Summary: A chocolate chip cookie is a drop cookie that features chocolate chips or chocolate morsels as its distinguishing ingredient. Chocolate chip cookies are claimed to have originated in the United States in 1938, when Ruth Graves Wakefield chopped up a Nestlé semi-sweet chocolate bar and added the chopped chocolate to a cookie recipe; however, historical recipes for grated or chopped chocolate cookies exist prior to 1938 by various other authors.
Generally, the recipe starts with a dough composed of flour, butter, both brown and white sugar, semi-sweet chocolate chips, eggs, and vanilla. Variations on the recipe may add other types of chocolate, as well as additional ingredients such as nuts or oatmeal. There are also vegan versions with the necessary ingredient substitutions, such as vegan chocolate chips, vegan margarine, and egg substitutes. A chocolat

{'input': 'Tell me about a recipe to make cookies',
 'output': 'Here is a simple recipe for chocolate chip cookies:\n\nIngredients:\n\n* 2 1/4 cups all-purpose flour\n* 1 tsp baking soda\n* 1 tsp salt\n* 1 cup (2 sticks) unsalted butter, at room temperature\n* 3/4 cup white granulated sugar\n* 3/4 cup brown sugar\n* 2 large eggs\n* 2 teaspoons pure vanilla extract\n* 2 cups semi-sweet chocolate chips\n\nInstructions:\n\n1. Preheat oven to 375°F (190°C). Line a baking sheet with parchment paper.\n2. In a medium bowl, whisk together the flour, baking soda, and salt. Set aside.\n3. In a large bowl, use an electric mixer to cream together the butter and sugars until light and fluffy, about 2-3 minutes.\n4. Beat in the eggs one at a time, followed by the vanilla extract.\n5. Gradually mix in the dry ingredients (flour mixture) until just combined, being careful not to overmix.\n6. Stir in the chocolate chips.\n7. Use a cookie scoop or tablespoon to drop rounded balls of dough onto the prepa

In [25]:
agent_executor.invoke(
    {
        "input": "what was my first query to you?"
    }
)



> Entering new AgentExecutor chain...
You didn't ask a query before this, our conversation just started.

> Finished chain.


{'input': 'what was my first query to you?',
 'output': "You didn't ask a query before this, our conversation just started."}